# Overview of Notebook
I don't believe scraping from the oxford dictionary is necessarily conforming to an ethical standard, so I will try and find a dataset to validate the myph datset other than the scraped oxford dictionary.

I'm primarily looking for a human curated dataset.

I'm going to the use the curated dataset from [DelphiForFun](http://www.delphiforfun.org/programs/Syllables.htm), by Gary D. Darby (God rest his soul). Darby used the mhyph dataset to generate a set of rules for syllabification. A limitation of his work is that his syllabification algorithm is not capable of syllabifying all English words, or pseudowords.

I'm going to use my syllabification network, trained on his dataset, for a more robust system of syllabification, one that is ethically sourced this time.

In [1]:
# 1
# imports
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re
import random
import os

In [10]:
# 2
# helper functions

def convert_to_hot(syl_word, split_token):
    hot = []
    i = 0
    while  i < len(syl_word):
        if i == len(syl_word) - 1:
            hot += [1]
            return hot
        if syl_word[i+1] == split_token:
            hot += [2]
            i += 2
        else:
            hot += [1]
            i += 1
    return hot

In [4]:
# 3
# environment constants

data_raw = 'data/Syllables.txt'
train_size = 8000
dev_size = 1000
test_size = 1000

output_directory = 'preprocessed'

split_token = '·'

In [8]:
# 4
# open the file, shuffle and sample

with open(data_raw, 'r', encoding='ISO-8859-1') as file:
    raw = [line.strip('\n').split('=') for line in file.readlines()]
    random.shuffle(raw)
    train = raw[:train_size]
    dev = raw[train_size: train_size + dev_size]
    test = raw[train_size + dev_size: train_size + dev_size + test_size]

In [11]:
# 5
# extracted train, dev, test, format into x,y with one hot
xtr = [x for x,y in train]
ytr = [convert_to_hot(y, split_token) for x,y in train]

xdev = [x for x,y in dev]
ydev = [convert_to_hot(y, split_token) for x,y in dev]

xte = [x for x,y in test]
yte = [convert_to_hot(y, split_token) for x,y in test]

In [14]:
# 6
# seting hyperparameters based on the data
x_raw = [x for x,_ in raw]

max_encoder_len = len(max(x_raw, key=len))
print(max_encoder_len)

16
